# Utils

> Configuration loading, saving, and conversion utilities

In [ ]:
#| default_exp core.utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, Any, Optional

## Configuration File Operations

In [ ]:
#| export
def load_config(
    schema_name: str,  # Name of the schema/configuration to load
    config_dir: Optional[Path] = None  # Directory where config files are stored
) -> Dict[str, Any]:  # Loaded configuration dictionary (empty dict if file doesn't exist)
    """Load saved configuration for a schema.
    
    Loads a JSON configuration file from the config directory.
    If the file doesn't exist, returns an empty dictionary.
    """
    if config_dir is None:
        from cjm_fasthtml_settings.core.config import DEFAULT_CONFIG_DIR
        config_dir = DEFAULT_CONFIG_DIR
        
    config_file = Path(config_dir) / f"{schema_name}.json"
    if config_file.exists():
        with open(config_file, "r") as f:
            return json.load(f)
    return {}

In [ ]:
#| export
def save_config(
    schema_name: str,  # Name of the schema/configuration to save
    config: Dict[str, Any],  # Configuration dictionary to save
    config_dir: Optional[Path] = None  # Directory where config files are stored
) -> bool:  # True if save succeeded, False otherwise
    """Save configuration for a schema.
    
    Saves a configuration dictionary as a JSON file in the config directory.
    Creates the config directory if it doesn't exist.
    """
    if config_dir is None:
        from cjm_fasthtml_settings.core.config import DEFAULT_CONFIG_DIR
        config_dir = DEFAULT_CONFIG_DIR
        
    try:
        config_dir = Path(config_dir)
        config_dir.mkdir(exist_ok=True, parents=True)
        
        config_file = config_dir / f"{schema_name}.json"
        with open(config_file, "w") as f:
            json.dump(config, f, indent=2)
        return True
    except Exception as e:
        print(f"Error saving config: {e}")
        return False

## Schema Utilities

In [ ]:
#| export
def get_default_values_from_schema(
    schema: Dict[str, Any]  # JSON Schema dictionary
) -> Dict[str, Any]:  # Dictionary of default values extracted from schema
    """Extract default values from a JSON schema.
    
    Iterates through the schema's properties and extracts any 'default' values.
    """
    values = {}
    properties = schema.get("properties", {})

    for prop_name, prop_schema in properties.items():
        if "default" in prop_schema:
            values[prop_name] = prop_schema["default"]

    return values

In [ ]:
# Example: Extract defaults from a schema
from cjm_fasthtml_settings.core.config import get_app_config_schema

schema = get_app_config_schema(app_title="Test App", server_port=8000)
defaults = get_default_values_from_schema(schema)

print("Default values:")
for key, value in defaults.items():
    print(f"  {key}: {value}")

Default values:
  app_title: Test App
  config_dir: configs
  auto_open_browser: True
  server_port: 8000
  server_host: 0.0.0.0
  debug_mode: False
  reload_on_change: False
  max_upload_size_mb: 100
  session_timeout_minutes: 60


In [ ]:
#| export
def get_config_with_defaults(
    schema_name: str,  # Name of the schema
    schema: Dict[str, Any],  # JSON Schema dictionary
    config_dir: Optional[Path] = None  # Directory where config files are stored
) -> Dict[str, Any]:  # Merged configuration with defaults and saved values
    """Get configuration with defaults merged with saved values.
    
    Loads saved configuration and merges it with schema defaults.
    Saved values take precedence over defaults.
    """
    saved_config = load_config(schema_name, config_dir)
    default_values = get_default_values_from_schema(schema)
    return {**default_values, **saved_config}

## Form Data Conversion

In [ ]:
#| export
def convert_form_data_to_config(
    form_data: dict,  # Raw form data from request
    schema: Dict[str, Any]  # JSON Schema for type conversion
) -> dict:  # Converted configuration dictionary
    """Convert form data to configuration dict based on schema.
    
    Handles type conversions for:
    - Boolean fields (checkboxes)
    - Integer and number fields
    - Array fields (comma-separated or Python list format)
    """
    config = dict(form_data)

    # Handle boolean fields (checkboxes)
    for prop_name, prop_schema in schema.get("properties", {}).items():
        if prop_schema.get("type") == "boolean":
            config[prop_name] = prop_name in config

    # Convert numeric fields
    for prop_name, prop_schema in schema.get("properties", {}).items():
        if prop_name in config:
            prop_type = prop_schema.get("type")
            value = config[prop_name]

            # Handle empty strings - convert to None for optional fields
            if value == "" or value is None:
                config[prop_name] = None
                continue

            if prop_type == "integer" or (isinstance(prop_type, list) and "integer" in prop_type):
                try:
                    config[prop_name] = int(value)
                except (ValueError, TypeError):
                    config[prop_name] = None
            elif prop_type == "number" or (isinstance(prop_type, list) and "number" in prop_type):
                try:
                    config[prop_name] = float(value)
                except (ValueError, TypeError):
                    config[prop_name] = None

    # Handle array fields
    for prop_name, prop_schema in schema.get("properties", {}).items():
        if prop_schema.get("type") == "array" and prop_name in config:
            value = config[prop_name]

            # If it's already a proper list, leave it
            if isinstance(value, list) and value and not isinstance(value[0], str):
                continue

            # If it's a string that looks like a Python list, parse it
            if isinstance(value, str):
                value = value.strip()
                if value.startswith('[') and value.endswith(']'):
                    try:
                        # Safely evaluate the list string
                        import ast
                        config[prop_name] = ast.literal_eval(value)
                    except (ValueError, SyntaxError):
                        # Fall back to comma-separated parsing
                        config[prop_name] = [item.strip() for item in value.split(',') if item.strip()]
                else:
                    # Treat as comma-separated values
                    config[prop_name] = [item.strip() for item in value.split(',') if item.strip()]
            elif isinstance(value, list):
                # If it's a list but contains string representations of lists
                if value and isinstance(value[0], str) and value[0].startswith('['):
                    # Join the parts and parse as a single list
                    joined = ''.join(value)
                    try:
                        import ast
                        config[prop_name] = ast.literal_eval(joined)
                    except (ValueError, SyntaxError):
                        # Keep as is if parsing fails
                        pass
            elif not isinstance(value, list):
                config[prop_name] = [value]

    return config

In [ ]:
# Example: Convert form data
from cjm_fasthtml_settings.core.config import get_app_config_schema

schema = get_app_config_schema()

# Simulate form data
form_data = {
    "app_title": "My App",
    "server_port": "8080",  # String from form
    "auto_open_browser": "on",  # Checkbox value
    "max_upload_size_mb": "250.5",  # String number
}

config = convert_form_data_to_config(form_data, schema)

print("Converted config:")
for key, value in config.items():
    print(f"  {key}: {value} ({type(value).__name__})")

Converted config:
  app_title: My App (str)
  server_port: 8080 (int)
  auto_open_browser: True (bool)
  max_upload_size_mb: 250.5 (float)
  debug_mode: False (bool)
  reload_on_change: False (bool)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()